## Project proposal
### Importing the dataset

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 16)
library(ggplot2)

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (770926916.py, line 4)

In [2]:
studenturl<- "https://archive.ics.uci.edu/static/public/320/student+performance.zip"
temp <- tempfile()
download.file(studenturl,temp)
studentzip <- unzip(temp, "student.zip")
studentfile <- unzip(studentzip,"student-por.csv")
student <- read_csv2(studentfile, col_names = TRUE)

student

NameError: name 'studenturl' is not defined

## Dataset background
## Possible questions from the dataset
- Proportion of students getting high to very high freetime after school in urban area in comparison of proportion of students getting high to very high freetime after school in rural area
- Mean final grade of students receiving extra support from school and family in comparison to students getting no support from school and family
- The mean number of absences for students in urban area in comparison to students in rural area. 
- Mean number of past class failures for students with family support in comparison to students with no family support. 3. mean final grade of students with a romantic relationship in comparison to the mean final grade of students with no romantic relationship.

## Citation

testing